#Load Data

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
#Import Data - GDP
gdp_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/GDP.csv")

In [ ]:
#Import data - World University Rankings
WUR_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/World University Rankings.csv")

#Inspect Data

In [ ]:
#Number of Rows/Columns for each dataset
print("GDP Dataset Shape:", gdp_df.shape)
print("World University Rankings Dataset Shape:", WUR_df.shape)

GDP Dataset Shape: (184, 18)
World University Rankings Dataset Shape: (2200, 14)


In [ ]:
#GDP Dataset Preview

gdp_df.head()

,Country,Population Rank,Growth Rate,World Percentage,Density,Land Area,2020 Population Rank,2020 World Percentage,2020 Growth Rate,Area,Capital City,Region,Subregion,Anthem,Government,GDP (IMF),GDP (UN),GDP Per Capita
0,Japan,11.0,-0.30%,1.62%,334.66,"364,555",11,1.62%,-0.30%,"377,930",Tokyo,Asia,Eastern Asia,Japan,Japan,$5.50 Tn,$4.94 Tn,"$43,450.14"
1,Mauritania,127.0,2.74%,0.06%,4.51,"1,030,700",127,0.06%,2.74%,"1,030,700",Nouakchott,Africa,Western Africa,NaN,NaN,$5.70 Bn,$4.67 Bn,"$1,225.25"
2,Cape Verde,172.0,1.10%,0.01%,137.86,"4,030",172,0.01%,1.10%,"4,033",Praia,Africa,Western Africa,NaN,NaN,$2.21 Bn,$1.64 Bn,"$3,971.32"
3,Tunisia,79.0,1.06%,0.15%,72.24,"155,360",79,0.15%,1.06%,"163,610",Tunis,Africa,Northern Africa,NaN,NaN,$35.15 Bn,$41.70 Bn,"$2,974.12"
4,Eritrea,133.0,1.41%,0.05%,30.16,"101,000",133,0.05%,1.41%,"117,600",Asmara,Africa,Eastern Africa,NaN,NaN,$8.12 Bn,$5.41 Bn,"$2,288.5"


In [ ]:
#World University Rankings (WUR) Dataset Preview

WUR_df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [ ]:
#Dataset Info
print("GDP Dataset Info:")
gdp_df.info()

print("\nWUR Dataset Info:")
WUR_df.info()

GDP Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                184 non-null    object 
 1   Population Rank        182 non-null    float64
 2   Growth Rate            182 non-null    object 
 3   World Percentage       182 non-null    object 
 4   Density                184 non-null    float64
 5   Land Area              182 non-null    object 
 6   2020 Population Rank   184 non-null    int64  
 7   2020 World Percentage  184 non-null    object 
 8   2020 Growth Rate       184 non-null    object 
 9   Area                   184 non-null    object 
 10  Capital City           184 non-null    object 
 11  Region                 184 non-null    object 
 12  Subregion              184 non-null    object 
 13  Anthem                 26 non-null     object 
 14  Government             78 non-null     o

#Clean Data

## Change Country Names

In [ ]:
# Unique countries in GDP data
gdp_countries = set(gdp_df['Country'].unique())

# Unique countries in University Rankings data
wur_countries = set(WUR_df['country'].unique())

# Check differences
country_diffs = wur_countries - gdp_countries
print("Mismatched Countries in WUR (not found in GDP):")
print(sorted(country_diffs))

Mismatched Countries in WUR (not found in GDP):
['Hong Kong', 'Slovak Republic', 'USA']


In [ ]:
#Check if Hong Kong is in GDP
print("Hong Kong SAR" in gdp_df['Country'].values)
print("Hong Kong" in gdp_df['Country'].values)

False
False


In [ ]:
#Drop Hong Kong
WUR_df = WUR_df[WUR_df['country'] != "Hong Kong"]

In [ ]:
#Country Mapping

country_mapping = {
    "USA": "United States",
    "Slovak Republic": "Slovakia"
}

In [ ]:
#Change Country Names WUR_df

WUR_df['country'] = WUR_df['country'].replace(country_mapping)

In [ ]:
#Verify if all countries now match

mismatches_after_mapping = set(WUR_df['country'].unique()) - set(gdp_df['Country'].unique())
print("Remaining mismatches:", mismatches_after_mapping)

Remaining mismatches: set()


##Alter Data Types

In [ ]:
#Clean GDP/IMF

def convert_gdp(value):
    if 'Tn' in value:
        return float(value.replace('$', '').replace('Tn', '').replace(',', '').strip()) * 1_000_000_000_000
    elif 'Bn' in value:
        return float(value.replace('$', '').replace('Bn', '').replace(',', '').strip()) * 1_000_000_000
    elif 'Mn' in value:
        return float(value.replace('$', '').replace('Mn', '').replace(',', '').strip()) * 1_000_000
    else:
        return None

gdp_df['GDP_IMF_clean'] = gdp_df['GDP (IMF)'].apply(convert_gdp)

In [ ]:
#Clean GDP Per Capita

def clean_capita(value):
    try:
        return float(value.replace('$', '').replace(',', '').strip())
    except:
        return None

gdp_df['GDP_Per_Capita_clean'] = gdp_df['GDP Per Capita'].apply(clean_capita)

In [ ]:
gdp_df[['Country', 'GDP (IMF)', 'GDP_IMF_clean', 'GDP Per Capita', 'GDP_Per_Capita_clean']].head()

,Country,GDP (IMF),GDP_IMF_clean,GDP Per Capita,GDP_Per_Capita_clean
0,Japan,$5.50 Tn,5.500000e+12,"$43,450.14",43450.14
1,Mauritania,$5.70 Bn,5.700000e+09,"$1,225.25",1225.25
2,Cape Verde,$2.21 Bn,2.210000e+09,"$3,971.32",3971.32
3,Tunisia,$35.15 Bn,3.515000e+10,"$2,974.12",2974.12
4,Eritrea,$8.12 Bn,8.120000e+09,"$2,288.5",2288.50


In [ ]:
gdp_df.head()

,Country,Population Rank,Growth Rate,World Percentage,Density,Land Area,2020 Population Rank,2020 World Percentage,2020 Growth Rate,Area,Capital City,Region,Subregion,Anthem,Government,GDP (IMF),GDP (UN),GDP Per Capita,GDP_IMF_clean,GDP_Per_Capita_clean
0,Japan,11.0,-0.30%,1.62%,334.66,"364,555",11,1.62%,-0.30%,"377,930",Tokyo,Asia,Eastern Asia,Japan,Japan,$5.50 Tn,$4.94 Tn,"$43,450.14",5.500000e+12,43450.14
1,Mauritania,127.0,2.74%,0.06%,4.51,"1,030,700",127,0.06%,2.74%,"1,030,700",Nouakchott,Africa,Western Africa,NaN,NaN,$5.70 Bn,$4.67 Bn,"$1,225.25",5.700000e+09,1225.25
2,Cape Verde,172.0,1.10%,0.01%,137.86,"4,030",172,0.01%,1.10%,"4,033",Praia,Africa,Western Africa,NaN,NaN,$2.21 Bn,$1.64 Bn,"$3,971.32",2.210000e+09,3971.32
3,Tunisia,79.0,1.06%,0.15%,72.24,"155,360",79,0.15%,1.06%,"163,610",Tunis,Africa,Northern Africa,NaN,NaN,$35.15 Bn,$41.70 Bn,"$2,974.12",3.515000e+10,2974.12
4,Eritrea,133.0,1.41%,0.05%,30.16,"101,000",133,0.05%,1.41%,"117,600",Asmara,Africa,Eastern Africa,NaN,NaN,$8.12 Bn,$5.41 Bn,"$2,288.5",8.120000e+09,2288.50


In [ ]:
# 1. Clean percentage columns (% → float)
percentage_cols = ['Growth Rate', 'World Percentage', '2020 World Percentage', '2020 Growth Rate']

for col in percentage_cols:
    gdp_df[col + '_clean'] = gdp_df[col].str.replace('%', '', regex=False).astype(float)

# 2. Clean area columns (remove commas → int)
area_cols = ['Land Area', 'Area']
for col in area_cols:
    gdp_df[col + '_clean'] = gdp_df[col].astype(str).str.replace(',', '', regex=False).astype(float)

# 3. Drop mostly empty columns
gdp_df.drop(columns=['Anthem', 'Government'], inplace=True)

# Optional: Rename columns for clarity (not required but good practice)
gdp_df.rename(columns={
    'GDP_IMF_clean': 'GDP (IMF, USD)',
    'GDP_Per_Capita_clean': 'GDP Per Capita (USD)',
    'Land Area_clean': 'Land Area (km²)',
    'Area_clean': 'Total Area (km²)',
    'Growth Rate_clean': 'Growth Rate (%)',
    'World Percentage_clean': 'World Pop Share (%)',
    '2020 World Percentage_clean': '2020 Pop Share (%)',
    '2020 Growth Rate_clean': '2020 Growth Rate (%)'
}, inplace=True)

In [ ]:
gdp_df.head(3)

,Country,Population Rank,Growth Rate,World Percentage,Density,Land Area,2020 Population Rank,2020 World Percentage,2020 Growth Rate,Area,...,GDP (UN),GDP Per Capita,"GDP (IMF, USD)",GDP Per Capita (USD),Growth Rate (%),World Pop Share (%),2020 Pop Share (%),2020 Growth Rate (%),Land Area (km²),Total Area (km²)
0,Japan,11.0,-0.30%,1.62%,334.66,"364,555",11,1.62%,-0.30%,"377,930",...,$4.94 Tn,"$43,450.14",5.500000e+12,43450.14,-0.30,1.62,1.62,-0.30,364555.0,377930.0
1,Mauritania,127.0,2.74%,0.06%,4.51,"1,030,700",127,0.06%,2.74%,"1,030,700",...,$4.67 Bn,"$1,225.25",5.700000e+09,1225.25,2.74,0.06,0.06,2.74,1030700.0,1030700.0
2,Cape Verde,172.0,1.10%,0.01%,137.86,"4,030",172,0.01%,1.10%,"4,033",...,$1.64 Bn,"$3,971.32",2.210000e+09,3971.32,1.10,0.01,0.01,1.10,4030.0,4033.0


In [ ]:
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                184 non-null    object 
 1   Population Rank        182 non-null    float64
 2   Growth Rate            182 non-null    object 
 3   World Percentage       182 non-null    object 
 4   Density                184 non-null    float64
 5   Land Area              182 non-null    object 
 6   2020 Population Rank   184 non-null    int64  
 7   2020 World Percentage  184 non-null    object 
 8   2020 Growth Rate       184 non-null    object 
 9   Area                   184 non-null    object 
 10  Capital City           184 non-null    object 
 11  Region                 184 non-null    object 
 12  Subregion              184 non-null    object 
 13  GDP (IMF)              184 non-null    object 
 14  GDP (UN)               184 non-null    object 
 15  GDP Pe